In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#run on 7/25/2020 data

In [3]:
#loading channels sheet
from os import listdir
filepaths=["./channel/day=25/hour=17/"+f for f in listdir("./channel/day=25/hour=17/") if f.endswith('.csv')]
rawchan=[]
for filename in filepaths:
    temp=pd.read_csv(filename, header=0, sep="|")
    rawchan.append(temp)
df_channel=pd.concat(rawchan, axis=0, ignore_index=True)

In [4]:
#loading video sheets
rawvideo=[]
filepaths=["./video/day=25/hour=17/"+f for f in listdir("./video/day=25/hour=17/") if f.endswith(".csv")]
for filename in filepaths:
    temp=pd.read_csv(filename, header=0, sep="|", error_bad_lines=False)
    rawvideo.append(temp)
df_video=pd.concat(rawvideo, axis=0, ignore_index=True)

b'Skipping line 18: expected 16 fields, saw 17\n'
b'Skipping line 86: expected 16 fields, saw 17\n'
b'Skipping line 294: expected 16 fields, saw 17\n'
b'Skipping line 191: expected 16 fields, saw 17\n'
b'Skipping line 206: expected 16 fields, saw 17\n'
b'Skipping line 23: expected 16 fields, saw 17\n'


In [5]:
df_video.id.duplicated().sum()

45463

In [6]:
df_channel.commentCount.value_counts()

0    2645
Name: commentCount, dtype: int64

In [7]:
#check variables
print(df_channel.columns)
print(df_video.columns)

Index(['id', 'title', 'collectedTime', 'category', 'sub_category',
       'description', 'publishedAt', 'thumbnailUrl', 'country', 'viewCount',
       'commentCount', 'subscriberCount', 'hiddenSubscriberCount',
       'videoCount', 'keywords', 'unsubscribedTrailer'],
      dtype='object')
Index(['id', 'collectedTime', 'publishedAt', 'channelId', 'title',
       'description', 'thumbnailUrl', 'tags', 'category', 'defaultLanguage',
       'duration', 'caption', 'viewCount', 'likeCount', 'dislikeCount',
       'commentCount'],
      dtype='object')


In [8]:
#change id@channel to channelId for ease of use
df_channel=df_channel.rename(columns={"id":"channelId"})

In [9]:
#date and time conversion
df_channel["collectedTime"]=pd.to_datetime(df_channel.collectedTime)
df_channel["publishedAt"]=pd.to_datetime(df_channel.publishedAt)
df_video["collectedTime"]=pd.to_datetime(df_video.collectedTime)
df_video["publishedAt"]=pd.to_datetime(df_video.publishedAt)

In [10]:
#sum total like and dislike from each videos
like=df_video.groupby(["channelId"]).sum()#get sum
like=like.drop(["viewCount", "commentCount"], axis=1).reset_index()#drop pointless columns and reset index
like

,channelId,likeCount,dislikeCount
0,UC-8R1d-KQrZy_7-EvE_w9Nw,237.0,12.0
1,UC-9nTjw_qHCA693UA3jyDWw,314351.0,2162.0
2,UC-9oz6OofrZnaOUQ48-LavA,370668.0,20715.0
3,UC-AuHoBXFe2e6YQ3fcA2zFA,472.0,10.0
4,UC-BqPABOl7c7rR5EoJJZ0UA,951905.0,10621.0
...,...,...,...
976,UCzXNBo1PYNsqSrN19XK3hPw,4200.0,529.0
977,UCzacSE_ExLj6L5rXx2CakjQ,812979.0,12617.0
978,UCzdPXIkTEXBkbx9cQ9aYDJA,6508.0,486.0
979,UCzmGNr7bHavW8SUg1n4KIMQ,29.0,8.0


In [11]:
#merge calculated like/dislikes into chanel data set
df_channel=pd.merge(df_channel, like, how="outer")

In [12]:
#title and category @ channel
chtovid=df_channel[["channelId", "title", "category", "sub_category"]]#creating sub df
chtovid=chtovid.rename(columns={"channelId":"channelId", "title":"ch_title", "category":"ch_category", "sub_category":"ch_sub_category"})
chtovid

,channelId,ch_title,ch_category,ch_sub_category
0,UCkinYTS9IHqOEwR1Sze2JTw,SBS 뉴스,T,TSB
1,UC6erIDuvbOaAO-OT5rB2Xew,SBS Drama,T,TSB
2,UCt51IEo3ZxxOysVAG_ylR6w,찬이TV,M,MSB
3,UCmJ4DkPFPzjR2TkJ7lXSQWw,YoonRoot,M,MSB
4,UCPITXstDqqLYbBUiAVqYjfg,메이비TV,M,MSB
...,...,...,...,...
2640,UCdS79cFd4MdxlgLHA-YYZ_A,윤아라YoonAra,M,MDT
2641,UCftAJ5MwttYA4kLXpN87BDg,아리나arina,M,MDT
2642,UCMtfNzI-KhpZter1cHmZC6A,써니튜브SUNNYTUBE,M,MDT
2643,UCbHyZneFBvQFBGde5YzAh-g,소확행 - 소비는 확실한 행복 -,M,KDC


In [13]:
#merge above title and category to video
df_video=pd.merge(df_video, chtovid, how="inner")

In [14]:
#parse date out of publishedAt from df_video
df_video["pubDate"]=df_video.publishedAt.astype(str).str[:-15]
df_video["pubDate"]=pd.to_datetime(df_video.pubDate)
df_video.head()

,id,collectedTime,publishedAt,channelId,title,description,thumbnailUrl,tags,category,defaultLanguage,duration,caption,viewCount,likeCount,dislikeCount,commentCount,ch_title,ch_category,ch_sub_category,pubDate
0,EPGtZlCxs14,2020-07-25 23:24:49,2020-01-16 09:41:20+00:00,UCzmGNr7bHavW8SUg1n4KIMQ,청송사랑화폐 홍보영상(2019년),"주왕산, 온천, 약수탕, 청송꽃돌 등 온갖 자연의 신비가 어우러진 청송 국가지질공원...",https://i.ytimg.com/vi/EPGtZlCxs14/default.jpg,청송사과주왕산주산지청송사과국가지질공원사과축제,19,ko,PT3M50S,False,73.0,2.0,0.0,NaN,청송군청,P,PGB,2020-01-16
1,JA8vgEBWWuY,2020-07-25 23:24:49,2019-12-02 06:01:01+00:00,UCzmGNr7bHavW8SUg1n4KIMQ,청송군 홍보영상_산소카페 청송군(2019년),"주왕산, 온천, 약수탕, 청송꽃돌 등 온갖 자연의 신비가 어우러진 청송 국가지질공원...",https://i.ytimg.com/vi/JA8vgEBWWuY/default.jpg,청송사과주왕산주산지청송사과국가지질공원사과축제,19,ko,PT8M46S,False,238.0,5.0,0.0,NaN,청송군청,P,PGB,2019-12-02
2,3_30R9TMNiI,2020-07-25 23:24:49,2019-11-27 04:21:53+00:00,UCzmGNr7bHavW8SUg1n4KIMQ,청송 유네스코세계지질공원 홍보 영상(2019),"주왕산, 온천, 약수탕, 청송꽃돌 등 온갖 자연의 신비가 어우러진 청송 국가지질공원...",https://i.ytimg.com/vi/3_30R9TMNiI/default.jpg,청송사과주왕산주산지청송사과국가지질공원사과축제,19,ko,PT2M45S,False,2201.0,6.0,0.0,NaN,청송군청,P,PGB,2019-11-27
3,0C5r3mYM95o,2020-07-25 23:24:49,2019-09-30 06:41:53+00:00,UCzmGNr7bHavW8SUg1n4KIMQ,2019 청송 아이스클라이밍 월드컵 홍보영상,"주왕산, 온천, 약수탕, 청송꽃돌 등 온갖 자연의 신비가 어우러진 청송 국가지질공원...",https://i.ytimg.com/vi/0C5r3mYM95o/default.jpg,청송사과주왕산주산지청송사과국가지질공원사과축제산소카페,19,ko,PT1M3S,False,22.0,1.0,0.0,NaN,청송군청,P,PGB,2019-09-30
4,-CW95pAEf-U,2020-07-25 23:24:49,2018-09-20 11:10:28+00:00,UCzmGNr7bHavW8SUg1n4KIMQ,윤경희 청송군수 2018 추석 인사,"주왕산, 온천, 약수탕, 청송꽃돌 등 온갖 자연의 신비가 어우러진 청송 유네스코세계...",https://i.ytimg.com/vi/-CW95pAEf-U/default.jpg,청송사과주왕산주산지청송사과유네스코세계지질공원사과축제국제슬로시티아이스클라이밍월드컵,19,ko,PT1M28S,False,73.0,1.0,0.0,NaN,청송군청,P,PGB,2018-09-20


In [15]:
#get recent 3-month data
df3M=df_video.set_index("pubDate").sort_index().last("3M")
df3M.reset_index()
df3M

,id,collectedTime,publishedAt,channelId,title,description,thumbnailUrl,tags,category,defaultLanguage,duration,caption,viewCount,likeCount,dislikeCount,commentCount,ch_title,ch_category,ch_sub_category
pubDate,,,,,,,,,,,,,,,,,,,
2020-05-01,0zUyANZKLQk,2020-07-25 22:42:52,2020-05-01 09:31:01+00:00,UCfWZnsdfiKvOyHDGfzMT3Cg,[어쩌다 하루] 청년들에게 따뜻한 위로가 되어주는 신부님의 김치찌개 20200501,★★★More ＂ordinaryday＂ clips are available★★★ ...,https://i.ytimg.com/vi/0zUyANZKLQk/default.jpg,김치찌개신부신부님기부재능기부어쩌다 하루어쩌다하루어쩌다하루손정은송진우,24,ko,PT3M59S,false,26.0,0.0,0.0,0.0,MBClife,T,TMC
2020-05-01,c4b9R6w2AOQ,2020-07-25 20:29:14,2020-05-01 11:21:21+00:00,UCeLPm9yH_a_QH8n6445G-Ow,[K-Fancam] (여자)아이들 슈화 '사랑해(Luv U)' ((G)I-DLE S...,#(G)I-DLE_SHUHUA #Fancam More from #KBSKPOP ...,https://i.ytimg.com/vi/c4b9R6w2AOQ/default.jpg,KBSKPOP뮤직KBSKPOPK-POP케이팝kpopKBSKpopk-choreoCho...,10,ko,PT3M54S,False,18799.0,1203.0,15.0,89.0,KBS Kpop,T,TKB
2020-05-01,eM2IWeECV2M,2020-07-25 22:37:57,2020-05-01 02:00:09+00:00,UCG5bAssl2H0wjLG4BEv5ScQ,[해를 품은 달] 14회 l “거짓이다” 한가인이 무녀임을 믿지 못하는 김수현 ＜드...,★매주 목금 오전 11시에 만나요☆ 드라마에도 맛집이 있다! [드라맛집] #드라맛집...,https://i.ytimg.com/vi/eM2IWeECV2M/default.jpg,드라맛집해를품은달해를 품은 달훤연우양명한가인김수현정일우,24,ko,PT15M21S,False,24866.0,186.0,11.0,17.0,옛드 : 옛날 드라마 [드라맛집],T,TMC
2020-05-01,VBW0WGKCi3Q,2020-07-25 20:29:14,2020-05-01 10:21:19+00:00,UCeLPm9yH_a_QH8n6445G-Ow,[K-Fancam] 크래비티 태영 'Break all the Rules' (CRAV...,#CRAVITY_TAEYOUNG #Fancam More from #KBSKPOP...,https://i.ytimg.com/vi/VBW0WGKCi3Q/default.jpg,KBSKPOP뮤직KBSKPOPK-POP케이팝kpopKBSKpop음악choreogra...,10,ko,PT3M41S,False,3393.0,391.0,2.0,16.0,KBS Kpop,T,TKB
2020-05-01,6p2TBmngndI,2020-07-25 22:42:52,2020-05-01 01:59:51+00:00,UCfWZnsdfiKvOyHDGfzMT3Cg,"[기분 좋은 날] 치매! 당신에게 일어날 수 있는 일, 치매 오기 특히 쉬운 사람은...",★★★More 'Good Day' clips are available★★★ iMB...,https://i.ytimg.com/vi/6p2TBmngndI/default.jpg,치매치매 증상치매 원인고혈압당뇨기분 좋은 날,24,ko,PT3M22S,false,27.0,0.0,0.0,0.0,MBClife,T,TMC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-25,Txc_F6XlnSo,2020-07-25 17:48:28,2020-07-25 07:32:34+00:00,UCcQTRi69dsVYHN3exePtZ1A,"[제보영상] 고양시 폐차장 큰불, 검은 연기 하늘 뒤덮어 KBS뉴스(News)",오늘(25일) 오전 11시 40분쯤 경기도 고양시 일산서구의 한 폐차장에서 불이 났...,https://i.ytimg.com/vi/Txc_F6XlnSo/default.jpg,일산폐차장화재인명피해고양시불소방당국119KBS뉴스제보영상,25,ko,PT1M9S,False,190.0,15.0,1.0,4.0,KBS News,T,TKB
2020-07-25,JAy88PDxMIM,2020-07-25 20:37:02,2020-07-25 11:00:02+00:00,UCweOkPb1wVVH0Q0Tlj4a5Pw,"[MV] 싹쓰리 (유두래곤, 린다G, 비룡) _ Beach Again(다시 여기 바닷가)","[MV] 싹쓰리 (유두래곤, 린다G, 비룡) _ Beach Again(다시 여기 바...",24,ko,PT4M21S,false,12439,1851,23.0,212.0,NaN,NaN,1theK (원더케이),M,MSU
2020-07-25,1USyvLyja7Y,2020-07-25 17:48:28,2020-07-25 06:01:12+00:00,UCcQTRi69dsVYHN3exePtZ1A,[중앙방역대책본부] 코로나19 신규 확진자 113명…115일 만에 첫 세자리 - 7...,이라크에서 귀국한 근로자들과 러시아 선원들의 잇단 코로나19 확진으로 115일 만에...,https://i.ytimg.com/vi/1USyvLyja7Y/default.jpg,코로나19코로나신규확진자전염러시아선원해외유입해외kbs뉴스현장영상,25,ko,PT10M45S,False,466.0,32.0,2.0,15.0,KBS News,T,TKB


In [16]:
#extract number of videos from the last three months using above df
chanvid=df3M.channelId.value_counts().rename_axis("channelId").reset_index(name="3M_video")
chanvid=chanvid.fillna(value=0)

In [17]:
#3 month outcomes together
df3M_sum=df3M.groupby(["channelId"]).sum().reset_index()
df3M_sum=df3M_sum.rename(columns={"viewCount":"3M_viewCount", "likeCount":"3M_likeCount", "commentCount":"3M_commentCount", "dislikeCount":"3M_dislikeCount"})
df3M_sum=df3M_sum.fillna(value=0)
df3M_sum

,channelId,3M_viewCount,3M_likeCount,3M_dislikeCount,3M_commentCount
0,UC-8R1d-KQrZy_7-EvE_w9Nw,59039.0,196.0,12.0,27.0
1,UC-9nTjw_qHCA693UA3jyDWw,6827.0,465.0,7.0,119.0
2,UC-9oz6OofrZnaOUQ48-LavA,645573.0,14911.0,310.0,1388.0
3,UC-AuHoBXFe2e6YQ3fcA2zFA,4125.0,190.0,2.0,0.0
4,UC-BqPABOl7c7rR5EoJJZ0UA,2797055.0,104170.0,419.0,12426.0
...,...,...,...,...,...
851,UCzU1v5rL8paTs1herRUgOTg,1060.0,38.0,0.0,3.0
852,UCzW0FveoY3_sTgBjvpG7Kwg,10553.0,819.0,1.0,685.0
853,UCzXNBo1PYNsqSrN19XK3hPw,2493.0,67.0,1.0,7.0
854,UCzacSE_ExLj6L5rXx2CakjQ,64198.0,5277.0,12.0,440.0


In [18]:
#merge to df_channel
df_channel=pd.merge(df_channel, chanvid, how="outer")
df_channel=pd.merge(df_channel, df3M_sum, how="outer")
df_channel.head()

,channelId,title,collectedTime,category,sub_category,description,publishedAt,thumbnailUrl,country,viewCount,...,videoCount,keywords,unsubscribedTrailer,likeCount,dislikeCount,3M_video,3M_viewCount,3M_likeCount,3M_dislikeCount,3M_commentCount
0,UCkinYTS9IHqOEwR1Sze2JTw,SBS 뉴스,2020-07-25 17:40:42,T,TSB,"시청자와 함께 만들어 가는 뉴스, SBS 뉴스의 공식 유튜브 채널입니다.",2014-05-02 10:13:29+00:00,https://yt3.ggpht.com/a/AATXAJxqY68vNV1eQDp-pf...,KR,1303959116,...,84058,"SBSNEWS sbs sbs뉴스 korea KOREA korea news"" ""kor...",P2HHAuxe3QM,6552226.0,635819.0,5389.0,142959983.0,1013117.0,84934.0,756847.0
1,UC6erIDuvbOaAO-OT5rB2Xew,SBS Drama,2020-07-25 17:40:42,T,TSB,SBS 공식 드라마 채널 SBS Drama입니다! SBS의 드라마 프로그램을 클립별...,2011-02-21 06:03:56+00:00,https://yt3.ggpht.com/a/AATXAJxjSa6BNQWag0Y-dW...,KR,3955084431,...,338796,SBS Drama 드라마 다시보기 월화 수목 일일 주말,RJ6ceHZAshI,11164151.0,640730.0,452.0,26766897.0,266359.0,5875.0,22722.0
2,UCt51IEo3ZxxOysVAG_ylR6w,찬이TV,2020-07-25 17:40:42,M,MSB,안녕하세요. 오신여러분들 반갑습니다. 좋은 컨텐츠로 찾아뵙겠습니다!,2014-06-27 06:53:48+00:00,https://yt3.ggpht.com/a/AATXAJyA7a2KZAtO1cOnWp...,KR,190499520,...,1476,마인크래프트 상황극 로블록스 minecraft roblox,RzeaF5GPZII,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,UCmJ4DkPFPzjR2TkJ7lXSQWw,YoonRoot,2020-07-25 17:40:42,M,MSB,트위치 게임 스트리머 윤루트(YoonRoot)의 유튜브 채널입니다!,2009-10-03 07:54:56+00:00,https://yt3.ggpht.com/a/AATXAJwp4VzzB-oe6XAjaS...,KR,69377307,...,1254,"배그 하이라이트"" ""배틀그라운드 하이라이트"" 배그 윤루트 PUBG 배틀그라운드 ba...",-cXypjcg5xw,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,UCPITXstDqqLYbBUiAVqYjfg,메이비TV,2020-07-25 17:40:42,M,MSB,With Sandbox Network 펭귄 이글루 메이비의 유튜브 채널입니다. 트위...,2012-07-05 15:45:23+00:00,https://yt3.ggpht.com/a/AATXAJwRcu0fU0O2Kae_J5...,KR,51425547,...,1180,그림 구체관절인형 노래 마인크래프트,t9qux2AxkGs,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#create final dataset
df_channel_k=df_channel
df_channel_k["산출물수"]=df_channel["videoCount"]
df_channel_k["정기성"]=df_channel["3M_video"]
df_channel_k["정량개선도"]=(df_channel["3M_video"])/(df_channel["videoCount"])
df_channel_k["구독자수"]=df_channel.subscriberCount
df_channel_k["총조회수"]=df_channel.viewCount
df_channel_k["총좋아요수"]=df_channel.likeCount
df_channel_k["총싫어요수"]=df_channel.dislikeCount
df_channel_k["총댓글수"]=df_channel.commentCount
df_channel_k["MAU"]=(df_channel.viewCount)/(df_channel.subscriberCount)
df_channel_k["최근조회수"]=df_channel["3M_viewCount"]
df_channel_k["최근좋아요수"]=df_channel["3M_likeCount"]
df_channel_k["최근싫어요수"]=df_channel["3M_dislikeCount"]
df_channel_k["최근댓글수"]=df_channel["3M_commentCount"]
df_channel_k["조회수개선도"]=df_channel_k["최근조회수"]/df_channel_k["총조회수"]
df_channel_k["좋아요개선도"]=df_channel_k["최근좋아요수"]/df_channel_k["총좋아요수"]
df_channel_k["댓글수개선도"]=df_channel_k["최근댓글수"]/df_channel_k["총댓글수"]
df_channel_k["정성개선도"]=df_channel_k["조회수개선도"]+df_channel_k["좋아요개선도"]+df_channel_k["댓글수개선도"]
df_channel_k=df_channel_k.reset_index()
df_channel_k=df_channel_k.drop(['index','thumbnailUrl', 'country', 'viewCount', 'commentCount', 'subscriberCount', 'hiddenSubscriberCount', 'videoCount', 'keywords', 'unsubscribedTrailer', 'likeCount', 'dislikeCount', '3M_video', "3M_viewCount", "3M_likeCount", "3M_dislikeCount","3M_commentCount"], axis=1)
df_channel_k=df_channel_k.rename(columns={"channelId":"채널ID", "title":"채널명", "collectedTime":"수집일자", "category":"분류1", "sub_category":"분류2", "description":"채널설명", "publishedAt":"채널생성일","hiddenSubscriberCount":"구독자숨김"})
df_channel_k.columns

Index(['채널ID', '채널명', '수집일자', '분류1', '분류2', '채널설명', '채널생성일', '산출물수', '정기성',
       '정량개선도', '구독자수', '총조회수', '총좋아요수', '총싫어요수', '총댓글수', 'MAU', '최근조회수',
       '최근좋아요수', '최근싫어요수', '최근댓글수', '조회수개선도', '좋아요개선도', '댓글수개선도', '정성개선도'],
      dtype='object')

In [20]:
#"P" channels only
dfP_ch=df_channel_k[df_channel_k["분류1"]=="P"]
dfP_vid=df_video[(df_video.channelId.isin(dfP_ch["분류1"]))]
#"M" channels only
dfM_ch=df_channel_k[df_channel_k["분류1"]=="M"]
dfM_vid=df_video[(df_video.channelId.isin(dfM_ch["분류1"]))]
#"T" channels only
dfT_ch=df_channel_k[df_channel_k["분류1"]=="T"]
dfT_vid=df_video[(df_video.channelId.isin(dfT_ch["분류1"]))]
# df3MP=df3M[(df3M.channelId.isin(dfP_ch["분류1"]))]

In [21]:
#create standardized score
Tcols=['산출물수', '정기성', '정량개선도', '구독자수', '총조회수', '총좋아요수', '총싫어요수', '총댓글수', 'MAU', '최근조회수', '최근좋아요수', '최근싫어요수', '최근댓글수']#columns for T-score generation
for item in Tcols:
    item_Tscore=item+"_Tscore"
    df_channel_k[item_Tscore]=100+10*((df_channel_k[item] - df_channel_k[item].mean())/(df_channel_k[item].std()))#T-score=100+10z
df_channel_k.head()

,채널ID,채널명,수집일자,분류1,분류2,채널설명,채널생성일,산출물수,정기성,정량개선도,...,구독자수_Tscore,총조회수_Tscore,총좋아요수_Tscore,총싫어요수_Tscore,총댓글수_Tscore,MAU_Tscore,최근조회수_Tscore,최근좋아요수_Tscore,최근싫어요수_Tscore,최근댓글수_Tscore
0,UCkinYTS9IHqOEwR1Sze2JTw,SBS 뉴스,2020-07-25 17:40:42,T,TSB,"시청자와 함께 만들어 가는 뉴스, SBS 뉴스의 공식 유튜브 채널입니다.",2014-05-02 10:13:29+00:00,84058,5389.0,0.064110,...,102.174050,107.204689,103.591951,116.731778,NaN,NaN,151.117933,109.053704,134.572203,169.174131
1,UC6erIDuvbOaAO-OT5rB2Xew,SBS Drama,2020-07-25 17:40:42,T,TSB,SBS 공식 드라마 채널 SBS Drama입니다! SBS의 드라마 프로그램을 클립별...,2011-02-21 06:03:56+00:00,338796,452.0,0.001334,...,115.103331,123.957195,107.192851,116.877430,NaN,NaN,107.522355,101.197689,100.741501,100.325217
2,UCt51IEo3ZxxOysVAG_ylR6w,찬이TV,2020-07-25 17:40:42,M,MSB,안녕하세요. 오신여러분들 반갑습니다. 좋은 컨텐츠로 찾아뵙겠습니다!,2014-06-27 06:53:48+00:00,1476,NaN,NaN,...,100.112507,100.168719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,UCmJ4DkPFPzjR2TkJ7lXSQWw,YoonRoot,2020-07-25 17:40:42,M,MSB,트위치 게임 스트리머 윤루트(YoonRoot)의 유튜브 채널입니다!,2009-10-03 07:54:56+00:00,1254,NaN,NaN,...,99.377981,99.403345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,UCPITXstDqqLYbBUiAVqYjfg,메이비TV,2020-07-25 17:40:42,M,MSB,With Sandbox Network 펭귄 이글루 메이비의 유튜브 채널입니다. 트위...,2012-07-05 15:45:23+00:00,1180,NaN,NaN,...,99.195364,99.289908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_channel_k.columns

Index(['채널ID', '채널명', '수집일자', '분류1', '분류2', '채널설명', '채널생성일', '산출물수', '정기성',
       '정량개선도', '구독자수', '총조회수', '총좋아요수', '총싫어요수', '총댓글수', 'MAU', '최근조회수',
       '최근좋아요수', '최근싫어요수', '최근댓글수', '조회수개선도', '좋아요개선도', '댓글수개선도', '정성개선도',
       '산출물수_Tscore', '정기성_Tscore', '정량개선도_Tscore', '구독자수_Tscore',
       '총조회수_Tscore', '총좋아요수_Tscore', '총싫어요수_Tscore', '총댓글수_Tscore',
       'MAU_Tscore', '최근조회수_Tscore', '최근좋아요수_Tscore', '최근싫어요수_Tscore',
       '최근댓글수_Tscore'],
      dtype='object')

In [23]:
#calculate scores using T-scores
df_channel_k["MAU_Tscore"]=(df_channel_k["총조회수_Tscore"])/(df_channel_k["구독자수_Tscore"])
df_channel_k["정량개선도_Tscore"]=(df_channel_k["정기성_Tscore"])/(df_channel_k["산출물수_Tscore"])
df_channel_k["조회수개선도_Tscore"]=df_channel_k["최근조회수_Tscore"]/df_channel_k["총조회수_Tscore"]
df_channel_k["좋아요개선도_Tscore"]=df_channel_k["최근좋아요수_Tscore"]/df_channel_k["총좋아요수_Tscore"]
df_channel_k["댓글수개선도_Tscore"]=df_channel_k["최근댓글수_Tscore"]/df_channel_k["총댓글수_Tscore"]

In [24]:
#code NA into 0
df_channel_k.fillna(0, inplace=True)
df_channel_k.columns

Index(['채널ID', '채널명', '수집일자', '분류1', '분류2', '채널설명', '채널생성일', '산출물수', '정기성',
       '정량개선도', '구독자수', '총조회수', '총좋아요수', '총싫어요수', '총댓글수', 'MAU', '최근조회수',
       '최근좋아요수', '최근싫어요수', '최근댓글수', '조회수개선도', '좋아요개선도', '댓글수개선도', '정성개선도',
       '산출물수_Tscore', '정기성_Tscore', '정량개선도_Tscore', '구독자수_Tscore',
       '총조회수_Tscore', '총좋아요수_Tscore', '총싫어요수_Tscore', '총댓글수_Tscore',
       'MAU_Tscore', '최근조회수_Tscore', '최근좋아요수_Tscore', '최근싫어요수_Tscore',
       '최근댓글수_Tscore', '조회수개선도_Tscore', '좋아요개선도_Tscore', '댓글수개선도_Tscore'],
      dtype='object')

In [25]:
df_channel_k["정성개선도_Tscore"]=df_channel_k["조회수개선도_Tscore"]+df_channel_k["좋아요개선도_Tscore"]+df_channel_k["댓글수개선도_Tscore"]

In [26]:
df_channel_k.to_csv("종합홍보지수.csv", sep="|")